#questions

#Define the problem

the outcome of a specific speed dating session is needed to be predicted based on the profile of two people, therefore a recommendation system is implemented to better match people in speed dating events. but the given data has a lot of missing values that must be handled to perform a good model


#What is the input?

according to the privided data we have features for each participant which will help us to know some information about Participants help in a prediction process.


#What is the output?

the two participants are matched or not matched based on the provided data.


#What data mining function is required?

binary classification function


#What could be the challenges?

The data have a large number of null values should be handled. in addition you should choose the best model for the classification problem

#What is the impact?

The recommendation system can select two people that perfectly matches each other based on their answers in the application

#What is an ideal solution?

handling the missing values in data perfectly and selecting the best hyperparameters that give high performance in  my case I noticed that the best model was the Xgbclassifier performed well

#Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?
A simple linear regression model without any activation function is not suitable for classification tasks because it does not produce binary output. In linear regression, the output is a continuous value that can take any value within a range. In contrast, in classification tasks, the output should be a binary value that represents the class membership of the input. while, perceptron and logistic regression models are specifically designed for binary classification tasks. The perceptron model uses a step function as the activation function, which maps the output to a binary value depending on whether the output is above or below a threshold. The logistic regression model uses the sigmoid function as the activation function, which maps the output to a probability value between 0 and 1, representing the likelihood of the input belonging to the positive class.

#What's a decision tree and how it is different to a logistic regression model?
A decision tree is a type of supervised machine learning algorithm that is used for classification and regression tasks. It is a tree-like model where each node represents a decision based on the value of a feature, and each branch represents the outcome of that decision.
he main difference between decision tree and logistic regression is that decision trees are non-parametric models, meaning that they do not make any assumptions about the functional form of the relationship between the input features and the output variable. In contrast, logistic regression is a parametric model, meaning that it assumes a linear relationship between the input features and the log-odds of the positive class.
In addition, decision trees can handle both categorical and continuous input features, while logistic regression is typically used for continuous input features only.

#What's the difference between grid search and random search?
Grid search and random search are both techniques used for hyperparameter tuning.
Grid search involves specifying a set of values for each hyperparameter of the model, and then exhaustively searching through all possible combinations of these values to find the combination that results in the best performance. While, random search  involves randomly sampling a set of values for each hyperparameter of the model, and then evaluating the performance of the model for each combination of these values.

#What's the difference between bayesian search and random search?
Bayesian search and random search are both techniques used for hyperparameter tuning.
Bayesian search, on the other hand, uses a probabilistic model to predict the performance of each hyperparameter configuration based on the results of previous evaluations. The model can be updated after each evaluation to incorporate the new information, and can guide the search towards promising regions of the hyperparameter space. while random search  involves randomly sampling a set of values for each hyperparameter of the model, and then evaluating the performance of the model for each combination of these values.

#preprocessing

In [1]:
#import libaries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
!pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 9.9 MB/s eta 0:00:00


In [2]:
#read data
df = pd.read_csv('/content/train.csv')

In [3]:
#display the first five rows 
df.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2583
1,1,14,1,3,10,2,NaN,8,8,63.0,...,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,6830
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4840
3,1,38,2,9,20,18,13.0,6,7,200.0,...,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,5508
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4828


In [4]:
#display the dimension of the data
df.shape

(5909, 192)

In [5]:
#drop duplicates but not found any duplicated rows
df = df.drop_duplicates()
df.shape

(5909, 192)

In [6]:

df.nunique()

gender         2
idg           44
condtn         2
wave          21
round         15
            ... 
sinc5_3        9
intel5_3       7
fun5_3        10
amb5_3         9
id          5909
Length: 192, dtype: int64

In [7]:
df.dtypes

gender        int64
idg           int64
condtn        int64
wave          int64
round         int64
             ...   
sinc5_3     float64
intel5_3    float64
fun5_3      float64
amb5_3      float64
id            int64
Length: 192, dtype: object

In [8]:
#iIwill take batches from the data to display null values every thirty columns
#print percentages of null values from 0:29 
print(((df.isna().sum().sort_values()[:30])/len(df))*100)

gender      0.000000
samerace    0.000000
match       0.000000
partner     0.000000
order       0.000000
id          0.000000
position    0.000000
round       0.000000
wave        0.000000
condtn      0.000000
idg         0.000000
pid         0.135387
race        0.761550
field       0.761550
race_o      0.812320
gaming      0.981554
imprace     0.981554
imprelig    0.981554
from        0.981554
movies      0.981554
theater     0.981554
goal        0.981554
hiking      0.981554
intel2_1    0.981554
tv          0.981554
sports      0.981554
concerts    0.981554
exercise    0.981554
dining      0.981554
museums     0.981554
dtype: float64


In [9]:
#print percentages of null values from 30:59 
print(((df.isna().sum().sort_values()[30:60])/len(df))*100)

art         0.981554
go_out      0.981554
tvsports    0.981554
fun2_1      0.981554
reading     0.981554
clubbing    0.981554
sinc2_1     0.981554
shopping    0.981554
yoga        0.981554
attr1_1     0.981554
sinc1_1     0.981554
intel1_1    0.981554
music       0.981554
attr2_1     0.981554
pf_o_att    0.998477
pf_o_sin    0.998477
pf_o_int    0.998477
field_cd    0.998477
age         1.066170
career      1.083094
shar2_1     1.100017
fun1_1      1.100017
age_o       1.100017
amb2_1      1.100017
pf_o_fun    1.116940
exphappy    1.184634
amb1_1      1.201557
date        1.218480
pf_o_amb    1.235404
amb3_1      1.370790
dtype: float64


In [10]:
#print percentages of null values from 60:89 
print(((df.isna().sum().sort_values()[60:90])/len(df))*100)

attr3_1      1.370790
sinc3_1      1.370790
fun3_1       1.370790
intel3_1     1.370790
pf_o_sha     1.404637
shar1_1      1.489254
career_c     1.692334
int_corr     1.844644
attr         2.420037
attr_o       2.589271
like         2.961584
like_o       3.096971
sinc         3.384667
intel        3.469284
sinc_o       3.536977
intel_o      3.723134
prob         3.841598
prob_o       3.976984
met          4.264681
fun          4.332374
fun_o        4.484684
met_o        4.653918
amb          8.529362
amb_o        8.664749
satis_2     10.949399
length      10.949399
sinc1_2     10.949399
fun1_2      10.949399
amb1_2      10.949399
intel1_2    10.949399
dtype: float64


In [11]:
#print percentages of null values from 90:119 
print(((df.isna().sum().sort_values()[90:120])/len(df))*100)

attr3_2     10.949399
sinc3_2     10.949399
intel3_2    10.949399
fun3_2      10.949399
amb3_2      10.949399
shar1_2     10.949399
attr1_2     11.135556
numdat_2    11.338636
zipcode     12.184803
shar        12.895583
shar_o      13.318666
match_es    13.978677
positin1    22.304959
amb4_1      22.778812
fun4_1      22.778812
intel4_1    22.778812
sinc4_1     22.778812
attr4_1     22.778812
shar4_1     23.066509
sinc2_2     31.240481
sinc4_2     31.240481
intel4_2    31.240481
fun4_2      31.240481
amb4_2      31.240481
shar4_2     31.240481
attr2_2     31.240481
intel2_2    31.240481
amb2_2      31.240481
shar2_2     31.240481
fun2_2      31.240481
dtype: float64


In [12]:
#print percentages of null values from 120:149 
print(((df.isna().sum().sort_values()[120:150])/len(df))*100)

attr4_2     31.240481
undergra    41.326790
attr5_1     41.496023
amb5_1      41.496023
fun5_1      41.496023
intel5_1    41.496023
sinc5_1     41.496023
sinc5_2     47.740734
intel5_2    47.740734
fun5_2      47.740734
amb5_2      47.740734
attr5_2     47.740734
income      48.434591
attr1_s     50.668472
sinc1_s     50.668472
intel1_s    50.668472
fun1_s      50.668472
amb1_s      50.668472
shar1_s     50.668472
attr3_s     51.937722
sinc3_s     51.937722
amb3_s      51.937722
intel3_s    51.937722
fun3_s      51.937722
intel1_3    52.546962
fun1_3      52.546962
shar1_3     52.546962
attr3_3     52.546962
sinc3_3     52.546962
intel3_3    52.546962
dtype: float64


In [13]:
#print percentages of null values from 150:179 
print(((df.isna().sum().sort_values()[150:180])/len(df))*100)

fun3_3      52.546962
amb3_3      52.546962
sinc1_3     52.546962
attr1_3     52.546962
amb1_3      52.546962
them_cal    52.546962
you_call    52.546962
date_3      52.546962
tuition     56.947030
mn_sat      62.176341
amb4_3      64.951768
amb2_3      64.951768
fun2_3      64.951768
intel2_3    64.951768
sinc2_3     64.951768
attr2_3     64.951768
shar4_3     64.951768
fun4_3      64.951768
intel4_3    64.951768
sinc4_3     64.951768
attr4_3     64.951768
shar7_3     76.087324
amb7_3      76.087324
fun7_3      76.087324
intel7_3    76.087324
sinc7_3     76.087324
attr7_3     76.087324
amb5_3      76.087324
intel5_3    76.087324
fun5_3      76.087324
dtype: float64


In [14]:
#print percentages of null values from 180:192 
print(((df.isna().sum().sort_values()[180:])/len(df))*100)

shar2_3     76.087324
sinc5_3     76.087324
attr5_3     76.087324
fun7_2      76.121171
attr7_2     76.121171
intel7_2    76.121171
shar7_2     76.239634
sinc7_2     76.476561
amb7_2      76.476561
expnum      78.304282
numdat_3    82.061262
num_in_3    92.215265
dtype: float64


In [15]:
# some simple transformation (always make a copy before assigning)
df2 = df.copy()

In [16]:
#drop columns that have 64% or more of null values
df = df.drop(['amb4_3','amb2_3','fun2_3','intel2_3','sinc2_3','attr2_3','shar4_3','fun4_3',
              'intel4_3','sinc4_3','attr4_3','shar7_3','amb7_3','fun7_3','intel7_3','sinc7_3','id',
              'attr7_3','amb5_3','intel5_3','fun5_3','shar2_3','sinc5_3','attr5_3','fun7_2','idg',
              'attr7_2','intel7_2','shar7_2','sinc7_2','amb7_2','expnum','numdat_3','num_in_3','pid'],axis=1)

In [17]:
df.shape

(5909, 157)

In [18]:
# checking types for later use
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5909 entries, 0 to 5908
Columns: 157 entries, gender to amb3_3
dtypes: float64(140), int64(9), object(8)
memory usage: 7.1+ MB


In [19]:
#assign features and target and split the data into train and validation
y = df[['match']]
X = df.drop(['match'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.1,shuffle=True,random_state=42)

In [20]:
# we extract numeric features and categorical features names

# select numeric features 
feat_num = list(X_train.select_dtypes(include=['float64', 'int64']))

# select categorical features
feat_cat = list(X_train.select_dtypes(include=['object']))

print('numeric features:', feat_num)
print('categorical features:', feat_cat)

numeric features: ['gender', 'condtn', 'wave', 'round', 'position', 'positin1', 'order', 'partner', 'int_corr', 'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out', 'career_c', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob', 'met', 'match_es', 'attr1_s', 'sinc1_s', 'intel1_s', 'fun1

#first model

In [22]:
#define pipe line 

# define a pipe line for numeric feature preprocessing
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())]
)

# define a pipe line for categorical feature preprocessing
transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)
# define the preprocessor  
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, feat_num),
        ('cat', transformer_categorical, feat_cat)
    ]
)

#model
xgb = XGBClassifier(silent=False, 
                      scale_pos_weight=1, 
                      colsample_bytree = 0.4,
                      learning_rate=0.02,
                       max_depth=11,
                      n_estimators=900,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      reg_alpha = 0.3,)

# combine the preprocessor with the model as a full pipeline
full_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', xgb)
    ]
)

In [23]:
full_pipline = full_pipline.fit(X_train,y_train)

[09:43:18] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



In [24]:
param_1 = {
     'model__n_estimators': [700,800,900],            
     'model__max_depth':[9,10,11] ,
     'model__learning_rate' :[0.01,0.02]   
     
}

#first trial

In [34]:
grid_search = GridSearchCV(
    full_pipline, param_1, cv=5, verbose=5, n_jobs=-1, scoring='roc_auc')

grid_search.fit(X_train, y_train)
print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[11:23:33] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

best score 0.8780042613259805
best score {'model__learning_rate': 0.01, 'model__max_depth': 9, 'model__n_estimators': 900}


**observations**

in this trial I used XGBClassifier Model and used grid search for tuning hyperparameters and it consumed about one hour and the best were
['model__learning_rate': 0.01, 'model__max_depth': 9, 'model__n_estimators': 900]
I used kfold cross validation set k=5
it was found that:
best score 0.8780042613259805  
score on leaderboard 0.88213 Public

#second trial

In [40]:
bayes_search = BayesSearchCV(full_pipline,param_1, cv=5, verbose=1, n_jobs=-1, scoring='roc_auc')

bayes_search.fit(X_train,y_train)

print('best score {}'.format(bayes_search.best_score_))

print('best score {}'.format(bayes_search.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[14:10:22] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

best score 0.8782391945408943
best score OrderedDict([('model__learning_rate', 0.010193572136616383), ('model__max_depth', 9), ('model__n_estimators', 900)])


**observations**

in this trial I used XGBClassifier Model and used bayes search for tuning hyperparameters and it consumed about more than one hour and the best were
[('model__learning_rate', 0.010193572136616383), ('model__max_depth', 9), ('model__n_estimators', 900)]
I used kfold cross validation set k=5
it was found that:
best score 0.8782391945408943  
score on leaderboard 0.88189 Public

#third trial

In [27]:
random_search = RandomizedSearchCV(
    full_pipline, param_1, cv=5, verbose=1, n_jobs=2, scoring='roc_auc')

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[10:09:36] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

best score 0.8777114724931824
best score {'model__n_estimators': 700, 'model__max_depth': 9, 'model__learning_rate': 0.01}


**observations**

in this trial I used XGBClassifier Model and used random search for tuning hyperparameters and it consumed about 47 min and the best were
['model__n_estimators': 700, 'model__max_depth': 9, 'model__learning_rate': 0.01]
I used kfold cross validation set k=5
it was found that:
best score 0.8777114724931824  
score on leaderboard 0.88213 Public

#second model

In [22]:
#define pipe line 

# define a pipe line for numeric feature preprocessing
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())]
)

# define a pipe line for categorical feature preprocessing
transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)
# define the preprocessor  
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, feat_num),
        ('cat', transformer_categorical, feat_cat)
    ]
)

#model
randm_forest = RandomForestClassifier()

# combine the preprocessor with the model as a full pipeline
full_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', randm_forest)
    ]
)

In [23]:
full_pipline = full_pipline.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [24]:
param_2 = {
     'model__n_estimators': [20, 30, 40],            
     'model__max_depth':[10, 20, 30] ,
     'preprocessor__num__imputer__strategy': ['mean','median']   
     
}

#fourth trial

In [25]:
grid_search = GridSearchCV(
    full_pipline, param_2, cv=5, verbose=5, n_jobs=-1, scoring='roc_auc')

grid_search.fit(X_train, y_train)
print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


best score 0.8413230042964335
best score {'model__max_depth': 10, 'model__n_estimators': 40, 'preprocessor__num__imputer__strategy': 'median'}


**observations**

in this trial I used random forest Model and used grid search for tuning hyperparameters and it consumed about one hour and the best were
['model__max_depth': 10, 'model__n_estimators': 40, 'preprocessor__num__imputer__strategy': 'median']
I used kfold cross validation set k=5
it was found that:
best score 0.8413230042964335  
score on leaderboard 0.84113 Public

#fifth trial

In [26]:
bayes_search = BayesSearchCV(full_pipline,param_2, cv=5, verbose=1, n_jobs=-1, scoring='roc_auc')

bayes_search.fit(X_train,y_train)

print('best score {}'.format(bayes_search.best_score_))

print('best score {}'.format(bayes_search.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


best score 0.8415618709768852
best score OrderedDict([('model__max_depth', 10), ('model__n_estimators', 40), ('preprocessor__num__imputer__strategy', 'mean')])


**observations**

in this trial I used random forest Model and used bayes search for tuning hyperparameters and it consumed about more than one hour and the best were
[('model__max_depth', 10), ('model__n_estimators', 40), ('preprocessor__num__imputer__strategy', 'mean')]
I used kfold cross validation set k=5
it was found that:
best score 0.8415618709768852  
score on leaderboard 0.83713 Public

#sixth trial

In [27]:
random_search = RandomizedSearchCV(
    full_pipline, param_2, cv=5, verbose=1, n_jobs=2, scoring='roc_auc')

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


best score 0.8397536088650712
best score {'preprocessor__num__imputer__strategy': 'mean', 'model__n_estimators': 40, 'model__max_depth': 10}


**observations**

in this trial I used random forest Model and used random search for tuning hyperparameters and it consumed about 47 min and the best were
['preprocessor__num__imputer__strategy': 'mean', 'model__n_estimators': 40, 'model__max_depth': 10]
I used kfold cross validation set k=5
it was found that:
best score 0.8397536088650712  
score on leaderboard 0.82831 Public